<a href="https://colab.research.google.com/github/Irajkiran/appAIChatPDF/blob/main/appChatPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain
!pip install sentence_transformers
!pip install langchain-huggingface
!pip install PyPDF2
!pip install faiss-cpu
!pip install langchain-community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.6/987.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.5/366.5 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [ ]:
import huggingface_hub
from huggingface_hub import login
from transformers import pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import PyPDF2
from PyPDF2 import PdfReader
import langchain
from huggingface_hub import login
from google.colab import userdata
from langchain_text_splitters import RecursiveCharacterTextSplitter
import langchain_community
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_huggingface.llms import HuggingFacePipeline


def extract_text(pdf_path):
  """
  Extracts text content from a PDF document.
  """
  with open(pdf_path, 'rb') as f:
    reader = PdfReader(f)
    text = ""
    for page in reader.pages:
      text += page.extract_text()

  return text

def get_pdf_chunks(text):
  text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )

  chunks = text_splitter.split_text(text=text)
  return chunks

def get_vector_store(text_chunks):
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-mpnet-base-v2')
  VectorStore = FAISS.from_texts(text_chunks,embeddings)
  VectorStore.save_local("faiss_index")

def get_conversational_chain():
  prompt_template = """
  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in the
  provided context just say, "answer is not available in the context", don't provide the worng answer.\n
  Context: \n{context}?\n
  Question: \n{question}\n

  Answer:
  """
    # Load Gemma model from Hugging Face
  _tokenhuggungface = userdata.get('HuggingFaceToken')

  login(token = _tokenhuggungface)
  # Load model directly
  tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
  model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

  # Wrap the Hugging Face model in a pipeline for text generation
  pipe = pipeline(
      "text-generation", model=model, tokenizer=tokenizer, max_length=2048
  )

  # Create a HuggingFacePipeline object, which is a Runnable
  llm = HuggingFacePipeline(pipeline=pipe)

  prompt = PromptTemplate(template=prompt_template, input_variables=["context","question"])
  chain = load_qa_chain(llm,chain_type="stuff", prompt=prompt)

  # Extract text from PDF
  text = extract_text(pdf_path)
  return chain

def user_input(user_question):
  embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-mpnet-base-v2')
  #db.save_local("faiss_index")
  new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization = True)
  docs = new_db.similarity_search(user_question)

  chain = get_conversational_chain()


  response = chain(
      {"input_documents":docs, "question": user_question}
      , return_only_outputs = True)

  print(response)


def chat_with_pdf(pdf_path):
  """
  Initiates a text-based chat interaction with the PDF content.
  """

  get_vector_store(get_pdf_chunks(extract_text(pdf_path)))

  while True:
    # User query
    _user_input = input()
    if _user_input.lower() == "quit":
      break

    # Print response
    print(f"PDF: {user_input(_user_input)}")

if __name__ == "__main__":
  # Specify your PDF path
  pdf_path = r"/content/RPA.pdf"

  # Start chat interaction
  chat_with_pdf(pdf_path)

  print("Chat session ended.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
